In [1]:
import os
import random
import pickle

import comet_ml
from comet_ml import Artifact

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.optim.lr_scheduler import CyclicLR, ReduceLROnPlateau

from teste import engine, utils
from common.CustomData import MRI_NII_2D, MRI_NII_3D
from common.data_setup import data_transforms, DOMAINS
from common.networks import DAMS, BaselineCNN

from torchinfo import summary

# Setup target device
device = "cuda:4" if torch.cuda.is_available() else "cpu"
print(device)

/root/anaconda3/envs/lapixdl/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/anaconda3/envs/lapixdl/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


cuda:4


In [2]:
def contar_arquivos_em_subpastas(caminho):
    for pasta, subpastas, arquivos in os.walk(caminho):
        if len(arquivos) != 0:
            try:
                DOMAIN, SLIT, CLASS = rf"{pasta}".split("/")[-3:]
                print(f"{DOMAIN}/{SLIT}/{CLASS} contém {len(arquivos)} arquivos")
            except:
                print(pasta.split("/"))


# Exemplo de uso:
caminho = r"data/ADNI1-T1-AD-CN/Image/Preprocess/6_step_nifti_2d"
contar_arquivos_em_subpastas(caminho)

GE/test/CN contém 2880 arquivos
GE/test/AD contém 1984 arquivos
GE/val/CN contém 2336 arquivos
GE/val/AD contém 1216 arquivos
GE/train/CN contém 8640 arquivos
GE/train/AD contém 5472 arquivos
Philips/test/CN contém 992 arquivos
Philips/test/AD contém 544 arquivos
Philips/val/CN contém 704 arquivos
Philips/val/AD contém 320 arquivos
Philips/train/CN contém 2688 arquivos
Philips/train/AD contém 1792 arquivos
Siemens/test/CN contém 2656 arquivos
Siemens/test/AD contém 1376 arquivos
Siemens/val/CN contém 1984 arquivos
Siemens/val/AD contém 1216 arquivos
Siemens/train/CN contém 7840 arquivos
Siemens/train/AD contém 4544 arquivos


In [3]:
import multiprocessing

multiprocessing.cpu_count()

224

In [4]:
# Setup hyperparameters
NUM_EPOCHS = 50
NUM_WORKES = 8
SOURCE = "GE"
TARGET_1 = "Philips"
TARGET_2 = "Siemens"
EXP_NUMBER = "DAMS-SGD-11"
config = {
    # Pick the algorithm:
    "algorithm": "grid",
    # Declare your hyperparameters:
    "parameters": {
        "lr": {"type": "discrete", "values": [0.001, 0.0009, 0.0007]},  # 0.001,
        "batch_size": {"type": "discrete", "values": [16, 32, 64]},
        "dropout": {"type": "discrete", "values": [0.2, 0.3, 0.4]},
        "momentum": {
            "type": "discrete",
            "values": [0.9],
        },  # [0.0005, 0.0001, 0.005, 0.001]
        "weight_decay": {
            "type": "discrete",
            "values": [0.03, 0.02],
        },  # [0, 0.01, 0.001] - [0.0001, 0.001, 0.01]
        "model": {
            "type": "categorical",
            "values": ["DAMS"],
        },
        # "transform": {"type": "categorical", "values": ["train", "train_alt"]},
        "source": {"type": "categorical", "values": [SOURCE]},
        "target_1": {"type": "categorical", "values": [TARGET_1]},
        "target_2": {"type": "categorical", "values": [TARGET_2]},
        "seed": {"type": "discrete", "values": [8]},
    },
    # Declare what to optimize, and how:
    "spec": {
        "metric": "val_auc",
        "objective": "maximize",
    },
    "name": f"SOURCE_{SOURCE}",
    "trials": 1,
}

In [5]:
def seed_everything(seed: int):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # g = torch.Generator()
    # g.manual_seed(0)
    # return g

In [6]:
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

# Initialize the Comet SDK
comet_ml.login(project_name=f"ADNI1-s-{SOURCE}-{EXP_NUMBER}")
opt = comet_ml.Optimizer(config=config, verbose=0)

for exp in opt.get_experiments():
    # exp.log_parameter("random_state", 42)
    print(f'Treinando esse modelo: {exp.get_parameter("model")}')
    print(
        f'Treinando com o source: {exp.get_parameter("source")}/target: {exp.get_parameter("target_1")}/{exp.get_parameter("target_2")}'
    )

    # artifact = Artifact("Engine", "Code")
    # artifact.add("teste/engine.py")

    # exp.log_artifact(artifact)

    # artifact = Artifact("Networks", "Code")
    # artifact.add("common/networks.py")

    # exp.log_artifact(artifact)

    # artifact = Artifact("DataSetup", "Code")
    # artifact.add("common/data_setup.py")

    # exp.log_artifact(artifact)

    MODEL = exp.get_parameter("model")
    SOURCE = exp.get_parameter("source")
    TARGET_1 = exp.get_parameter("target_1")
    TARGET_2 = exp.get_parameter("target_2")
    seed_everything(exp.get_parameter("seed"))

    train_data_custom = MRI_NII_2D.Dataset2D(
        domain=DOMAINS[f"ADNI1-{SOURCE}"],
        split="train",
        transform=data_transforms["train"],
    )
    val_data_custom = MRI_NII_3D.Dataset3D(
        domain=DOMAINS[f"ADNI1-{SOURCE}-3D"],
        split="val",
        transform=None,
    )
    test_data_custom = MRI_NII_3D.Dataset3D(
        domain=DOMAINS[f"ADNI1-{SOURCE}-3D"],
        split="test",
        transform=None,
    )

    test_data_target = MRI_NII_3D.Dataset3D(
        domain=DOMAINS[f"ADNI1-{TARGET_1}-3D"],
        split="test",
        transform=None,
    )

    test_data_target_2 = MRI_NII_3D.Dataset3D(
        domain=DOMAINS[f"ADNI1-{TARGET_2}-3D"],
        split="test",
        transform=None,
    )

    train_dataloader = DataLoader(
        dataset=train_data_custom,  # use custom created train Dataset
        batch_size=exp.get_parameter("batch_size"),  # how many samples per batch?
        num_workers=NUM_WORKES,  # how many subprocesses to use for data loading? (higher = more)
        shuffle=True,
        pin_memory=True,
        # generator=g,
    )  # shuffle the data?
    val_dataloader = DataLoader(
        dataset=val_data_custom,  # use custom created test Dataset
        batch_size=1,
        num_workers=NUM_WORKES,
        shuffle=False,
        pin_memory=True,
        # generator=g,
    )  # don't usually need to shuffle testing data
    test_dataloader = DataLoader(
        dataset=test_data_custom,  # use custom created test Dataset
        batch_size=1,
        num_workers=NUM_WORKES,
        shuffle=False,
        pin_memory=True,
        # generator=g,
    )  # don't usually need to shuffle testing data

    test_target = DataLoader(
        dataset=test_data_target,  # use custom created test Dataset
        batch_size=1,
        num_workers=NUM_WORKES,
        shuffle=False,
        pin_memory=True,
        # generator=g,
    )  # don't usually need to shuffle testing data

    test_target_2 = DataLoader(
        dataset=test_data_target_2,  # use custom created test Dataset
        batch_size=1,
        num_workers=NUM_WORKES,
        shuffle=False,
        pin_memory=True,
        # generator=g,
    )  # don't usually need to shuffle testing data
    print("Loading Dataloader")
    # Create model with help from model_builder.py
    model = DAMS(dropout=exp.get_parameter("dropout")).to(device)
    # model = BaselineCNN().to(device)

    if SOURCE == "GE":
        pos_weight = torch.tensor([5472 / 8640]).to(device)
        exp.log_parameter("pos_weight", 5472 / 8640)
    elif SOURCE == "Siemens":
        pos_weight = torch.tensor([4544 / 7840]).to(device)
        exp.log_parameter("pos_weight", 4544 / 7840)
    elif SOURCE == "Philips":
        pos_weight = torch.tensor([1792 / 2688]).to(device)
        exp.log_parameter("pos_weight", 1792 / 2688)
    loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = torch.optim.SGD(
        model.parameters(),
        lr=exp.get_parameter("lr"),
        weight_decay=exp.get_parameter("weight_decay"),
        momentum=0.9,
    )
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    # Start training with help from engine.py
    print("Start Train")
    result = engine.train(
        model=model,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
        test_dataloader=test_dataloader,
        test_target=test_target,
        test_target_2=test_target_2,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=None,
        epochs=NUM_EPOCHS,
        early_stop_patience=10,
        exp=exp,
        checkpoint_path=rf"teste/models/{exp.get_name()}.pth",
        device=device,
    )

COMET INFO: 1939833dab1c442488c802649c1cf6b9
COMET INFO: Using optimizer config: {'algorithm': 'grid', 'configSpaceSize': 54, 'endTime': None, 'id': '1939833dab1c442488c802649c1cf6b9', 'lastUpdateTime': None, 'maxCombo': 0, 'name': 'SOURCE_GE', 'parameters': {'batch_size': {'type': 'discrete', 'values': [16, 32, 64]}, 'dropout': {'type': 'discrete', 'values': [0.2, 0.3, 0.4]}, 'lr': {'type': 'discrete', 'values': [0.001, 0.0009, 0.0007]}, 'model': {'type': 'categorical', 'values': ['DAMS']}, 'momentum': {'type': 'discrete', 'values': [0.9]}, 'seed': {'type': 'discrete', 'values': [8]}, 'source': {'type': 'categorical', 'values': ['GE']}, 'target_1': {'type': 'categorical', 'values': ['Philips']}, 'target_2': {'type': 'categorical', 'values': ['Siemens']}, 'weight_decay': {'type': 'discrete', 'values': [0.03, 0.02]}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 0, 'metric': 'val_auc', 'minSampleSize': 100, 'objective': 'maximize', 'randomize': False, 'retryAssignLimit': 0, '

Treinando esse modelo: DAMS
Treinando com o source: GE/target: Philips/Siemens
Loading Dataloader
Start Train


  0%|          | 0/50 [00:00<?, ?it/s]

/root/anaconda3/envs/lapixdl/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/root/anaconda3/envs/lapixdl/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Test Loss: 0.5227, Accuracy: 76/111 (68.47%), AUC: 0.5422
Test Loss: 0.5243, Accuracy: 73/111 (65.77%), AUC: 0.5732
Test Loss: 0.5377, Accuracy: 73/111 (65.77%), AUC: 0.5746
Test Loss: 0.8146, Accuracy: 73/111 (65.77%), AUC: 0.4405
Test Loss: 0.7356, Accuracy: 73/111 (65.77%), AUC: 0.5930
Test Loss: 0.9158, Accuracy: 73/111 (65.77%), AUC: 0.5281
Test Loss: 0.7456, Accuracy: 73/111 (65.77%), AUC: 0.5112
Test Loss: 0.9370, Accuracy: 73/111 (65.77%), AUC: 0.5750
Test Loss: 0.9055, Accuracy: 73/111 (65.77%), AUC: 0.6168
Test Loss: 1.1032, Accuracy: 73/111 (65.77%), AUC: 0.5519
Test Loss: 1.1158, Accuracy: 73/111 (65.77%), AUC: 0.5570
Test Loss: 1.0111, Accuracy: 73/111 (65.77%), AUC: 0.4048
Test Loss: 1.1238, Accuracy: 73/111 (65.77%), AUC: 0.5671
Test Loss: 1.0990, Accuracy: 73/111 (65.77%), AUC: 0.5274
Test Loss: 0.8625, Accuracy: 73/111 (65.77%), AUC: 0.6164
Test Loss: 1.1374, Accuracy: 73/111 (65.77%), AUC: 0.4838
Test Loss: 1.0239, Accuracy: 73/111 (65.77%), AUC: 0.4322
Test Loss: 1.0

...

Treinando esse modelo: DAMS
Treinando com o source: GE/target: Philips/Siemens
Loading Dataloader
Start Train


  0%|          | 0/50 [00:00<?, ?it/s]

/root/anaconda3/envs/lapixdl/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/root/anaconda3/envs/lapixdl/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Test Loss: 0.5291, Accuracy: 74/111 (66.67%), AUC: 0.6269
Test Loss: 0.5316, Accuracy: 38/111 (34.23%), AUC: 0.3955
Test Loss: 0.5259, Accuracy: 73/111 (65.77%), AUC: 0.4412
Test Loss: 0.5307, Accuracy: 43/111 (38.74%), AUC: 0.4492
Test Loss: 0.5273, Accuracy: 44/111 (39.64%), AUC: 0.6406
Test Loss: 0.6863, Accuracy: 65/111 (58.56%), AUC: 0.6218
Test Loss: 0.5368, Accuracy: 73/111 (65.77%), AUC: 0.5068
Test Loss: 0.5597, Accuracy: 73/111 (65.77%), AUC: 0.4311
Test Loss: 0.5786, Accuracy: 73/111 (65.77%), AUC: 0.4704
Test Loss: 0.6378, Accuracy: 73/111 (65.77%), AUC: 0.5252
Test Loss: 0.6360, Accuracy: 73/111 (65.77%), AUC: 0.5595
Test Loss: 0.6784, Accuracy: 73/111 (65.77%), AUC: 0.5025
Test Loss: 0.7851, Accuracy: 73/111 (65.77%), AUC: 0.6568
Test Loss: 0.7273, Accuracy: 73/111 (65.77%), AUC: 0.5703
Test Loss: 0.7342, Accuracy: 73/111 (65.77%), AUC: 0.6518
Test Loss: 0.7064, Accuracy: 73/111 (65.77%), AUC: 0.5148
Test Loss: 0.7329, Accuracy: 73/111 (65.77%), AUC: 0.6763
Test Loss: 0.7

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : thoughtless_dune_6673
COMET INFO:     url                   : https://www.comet.com/derick-amontagna/adni1-s-ge-dams-sgd-11/b97e6369745340569503cb49eb14af7a
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     epoch [27]         : (0, 26)
COMET INFO:     loss [597]         : (0.0339956171810627, 0.5745999813079834)
COMET INFO:     test_acc           : 59.21052631578947
COMET INFO:     test_acc_target    : 64.58333333333333
COMET INFO:     test_acc_target_2  : 65.87301587301587
COMET INFO:     test_auc           : 0.7267025113105774
COMET INFO:     test_auc_target    : 0.6622390747070312
COMET INFO:     test_auc_target_2  : 0.5671056509017944
COM